In [1]:
import tensorflow as tf
import numpy as np
import urllib
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-09-09 21:14:36.328094: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-09 21:14:36.333041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-09 21:14:36.333055: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

sentences = []
labels = []
# YOUR CODE HERE
with open('sarcasm.json', 'r') as f:
    data = json.load(f)

for row in data:
    sentences.append(row['headline'])
    labels.append(row['is_sarcastic'])

In [4]:
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

sentence_stop = []
for sentence in sentences:
    sentence_stop.append(' '.join([word for word in sentence.split() if word not in stopwords]))

In [6]:
# Tokenize and pad

train_sentences = sentences[:training_size]
train_labels = labels[:training_size]
train_labels = np.array(train_labels)

val_sentences = sentences[training_size:]
val_labels = labels[training_size:]
val_labels = np.array(val_labels)

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

training_sequences = tokenizer.texts_to_sequences(train_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [8]:
filters = 128
kernel_size = 5
dense_dim = 6

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Conv1D(filters, kernel_size, activation='relu'),
    # tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    # tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

es = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)

model.fit(training_padded,
          train_labels,
          epochs=20,
          validation_data=(val_padded, val_labels),
          callbacks=[es])

Epoch 1/20
625/625 [==============================] - 34s 51ms/step - loss: 0.4842 - accuracy: 0.7504 - val_loss: 0.3937 - val_accuracy: 0.8156
Epoch 2/20
625/625 [==============================] - 34s 55ms/step - loss: 0.3715 - accuracy: 0.8332 - val_loss: 0.3816 - val_accuracy: 0.8247
Epoch 3/20
625/625 [==============================] - 44s 70ms/step - loss: 0.3462 - accuracy: 0.8443 - val_loss: 0.3778 - val_accuracy: 0.8307
Epoch 4/20
625/625 [==============================] - 44s 71ms/step - loss: 0.3260 - accuracy: 0.8565 - val_loss: 0.3807 - val_accuracy: 0.8308
Epoch 5/20
625/625 [==============================] - 44s 71ms/step - loss: 0.3109 - accuracy: 0.8644 - val_loss: 0.3761 - val_accuracy: 0.8334
Epoch 6/20
625/625 [==============================] - 44s 71ms/step - loss: 0.2992 - accuracy: 0.8712 - val_loss: 0.3888 - val_accuracy: 0.8308
Epoch 7/20
625/625 [==============================] - 44s 71ms/step - loss: 0.2942 - accuracy: 0.8745 - val_loss: 0.3895 - val_accuracy: